# Trabalho prático 2: Realidade Aumentada
Aluno: Daniel Ferreira Abadi<br>
Matrícula: 2018088062<br>
Link do vídeo: https://youtu.be/ua_MD8KiKMs

O objetivo deste trabalho prático é detectar e localizar alvos fiduciários em um vídeo e inserir objetos tridimensionais em cada um dos alvos. Os objetos renderizados foram cubos, setas nas orientações dos alvos e Pikachus, esses deveriam ter rotações determinadas pelo alvo a que correspondem. Tudo isso foi feito em "tempo real", não havendo nenhum pré processamento.

Inicialmente precisamos extrair alguns frames do vídeo de entrada, eles foram adquiridos usando o seguinte comando no terminal do Linux: ffmpeg -i "entrada.mp4 -r 1/0.5 $frame%03d.jpg". Com isso foram gerados 75 imagens, com nomes 001.jpg até 075.jpg, pegando de meio em meio segundo do vídeo. Tendo as imagens em posse, foram escolhidas quatro, com distâncias temporais relevantes e poses distintas, para realizar a calibração.

Focal Length:          fc = [ 418.41479   416.04663 ] +/- [ 17.81913   17.41925 ]<br>
Principal point:       cc = [ 308.24745   221.17720 ] +/- [ 8.52814   13.53687 ]<br>
Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +/- 0.00000 degrees <br>
Distortion:            kc = [ 0.10892   -0.35851   -0.00736   -0.00382  0.00000 ] +/- [ 0.09273   0.66327   0.00869   0.00714  0.00000 ] <br>
Pixel error:          err = [ 0.19553   0.21926 ]

Na célula abaixo temos as importações necessárias para a realização do trabalho.

In [1]:
import cv2
import numpy as np

from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

from PIL import Image

from objloader import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Para podermos realizar o trabalho é necessário possuir todos os frames do vídeo de entrada. Esses foram obtidos por meio das funções "cv2.VideoCapture", "isOpened" e "read", onde cada frame é adicionado a uma lista, que será utilizada globalmente por todo o trabalho. Definimos então outras variáveis globais, como a "frame_atual" (serve meramente para dizer qual frame estamos lidando), a variável "angle" (controla a rotação dos Pikachus) e os parâmetros intrínsecos da câmera. É importante observar que o vídeo e o modelo 3D do Pikachu devem estar na mesma pasta que o notebook.

Por fim, temos o carregamento do alvo fiduciário que temos que detectar no vídeo. Para auxiliar a função de correlação criada mais a frente, a imagem do alvo em questão foi convertida para escala de cinza e binarizada, dois dos passos que os frames do vídeo passaram. Com isso, podemos pegar as quinas do alvo, que serão utilizadas posteriormente na homografia, e criar uma lista de alvos com suas rotações, que será utilizada na função de correlação. A ordem dos elementos na lista é importante, visto que a primeira posição não representa uma rotação (imagem original), a segunda posição uma rotação de 90 graus, a terceira 180 graus e a última 270 graus, todos em sentido horário.

In [2]:
frames = []
video_cap = cv2.VideoCapture('entrada.mp4')
while(video_cap.isOpened()):
    ret, frame = video_cap.read()
    if ret == False:
        break
    frames.append(frame)
    
frame_atual = 0
angle = 1

fx = 418.41479
fy = 416.04663
cx = 308.24745
cy = 221.17720

alvo = cv2.imread('alvo.jpg')
alvo = cv2.cvtColor(alvo, cv2.COLOR_RGB2GRAY)
ret, binary_alvo = cv2.threshold(alvo, 100, 255, cv2.THRESH_BINARY)

quinas_alvo = [[0,0]]
quinas_alvo.append([0, binary_alvo.shape[1]-1])
quinas_alvo.append([binary_alvo.shape[0]-1, binary_alvo.shape[1]-1])
quinas_alvo.append([binary_alvo.shape[0]-1, 0])

lista_alvos = [binary_alvo]
lista_alvos.append(cv2.rotate(alvo, cv2.ROTATE_90_CLOCKWISE))
lista_alvos.append(cv2.rotate(alvo, cv2.ROTATE_180))
lista_alvos.append(cv2.rotate(alvo, cv2.ROTATE_90_COUNTERCLOCKWISE))

# Detecção de alvos
Abaixo temos um conjunto de quatro funções que localizam o alvo e detectam a orientação dele na cena. A função que coordena todo o processo é a "localiza_alvos", que recebe um frame do vídeo de entrada, chama as outras funções e retorna uma lista de quinas dos alvos no frame e suas respectivas orientações. Primeiro ela transforma o frame do vídeo para a escala de cinza, isso é feito para que possamos binarizar a imagem, mas antes aplicamos um filtro de passa baixa (Gaussiano), para suavizar os serrilhados e melhorar a detecção dos alvos [Fonte 1].

Possuindo o frame binarizado, podemos detectar as bordas por meio da função "cv2.Canny", que nos foi sugerida na especificação do trabalho [Fonte 2]. Agora que as bordas estão enfatizadas, utilizamos a função "cv2.findContours" para detectar polígonos. A função "procura_quinas" é chamada passando os polígonos encontrados, ela apenas verifica se os polígonos possuem quatro lados e se possuem uma área menor do que 12.000 e maior que 600 (valores obtidos através de experimentação) para não fazermos homografias desnecessárias, em relação a quadrados pequenos ou a borda do vídeo, e otimizar o processo [Fontes 3 e 4]. A lista de quinas dos polígonos que passarem por esse filtro são adicionados em uma lista, que é retornada.

Retornando à função principal, ela verifica se a lista de quinas recebida está vazia, retornando duas listas vazias em caso positivo. Caso contrário, para cada conjunto de quinas, de cada polígono, é feita a homografia e a transformação "warpPerspective", ambas as transformações tendo como origem o conjunto de slides da matéria e [Fonte 5]. A homografia é feita utilizando as quinas do alvo em questão do frame atual e as quinas dos alvos originais, previamente carregadas acima. Já a transformação "warpPerspective", utiliza o frame atual binarizado, a homografia e as dimensões da imagem do alvo original, gerando uma imagem de mesmo tamanho. 

Todas essas imagens são inseridas em uma lista, que será utilizada para fazer a correlação por meio da função "faz_correlacao", que recebe essa lista e a lista de quinas encontradas. Essa função apenas normaliza a matriz de cada alvo encontrado e transformado, por meio da função "normaliza_alvo", calcula a correlação com cada um dos alvos da lista de alvos, definida acima, e verifica se a correlação foi maior do que 0.7 (valor encontrado experimentalmente) [Fonte 6]. Caso seja maior, a lista de quinas é inserida em uma nova lista e um número que representa a rotação é adicionado na lista de orientações. Esse número pode ser 0, 90, 180 e 270 e é definido pela posição em que se encontra a lista dos alvos originais. Com isso temos duas listas, uma contendo uma lista de quinas e a outra contendo as orientações, que possuem índices em comum, ou seja, índice 0 das duas correspondem ao mesmo alvo do frame atual. Essas listas são retornadas como resultado final desse bloco de funções.

Fontes:<br>
1 - https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html <br>
2 - https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga04723e007ed888ddf11d9ba04e2232de<br>
3 - https://pyimagesearch.com/2016/02/08/opencv-shape-detection/<br>
4 - https://docs.opencv.org/4.x/dd/d49/tutorial_py_contour_features.html<br>
5 - https://learnopencv.com/homography-examples-using-opencv-python-c/<br>
6 - https://stackoverflow.com/questions/53436231/normalized-cross-correlation-in-python<br>

In [3]:
def procura_quinas(contornos):
    quinas = []
    for c in contornos:
        epsilon = 0.02 * cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, epsilon, True)
        if len(approx) == 4:
            if cv2.contourArea(c) < 120000 and cv2.contourArea(c) > 600:
                quinas.append(np.array([[approx[0][0][0], approx[0][0][1]],
                          [approx[1][0][0], approx[1][0][1]],
                          [approx[2][0][0], approx[2][0][1]],
                          [approx[3][0][0], approx[3][0][1]]]))
    return quinas

def normaliza_alvo(alvo):
    mean_data = np.mean(alvo)
    std_data = np.std(alvo, ddof=1)
    return (alvo-mean_data)/(std_data)

def faz_correlacao(alvos_transformados, quinas):
    novas_quinas = []
    orientacao = []
    for i in range(len(alvos_transformados)):
        size = alvos_transformados[0].size - 1
        alvo_normalizado = normaliza_alvo(alvos_transformados[i])
        for j in range(len(lista_alvos)):
            cor = ((1.0/(size)) * np.sum(alvo_normalizado * normaliza_alvo(lista_alvos[j])))
            if cor > 0.7:
                if j == 0:
                    orientacao.append(0)
                elif j == 1:
                    orientacao.append(90)
                elif j == 2:
                    orientacao.append(180)
                else:
                    orientacao.append(270)
                novas_quinas.append(quinas[i])
    return novas_quinas, orientacao

def localiza_alvos(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame,(5,5),0)
    ret, binary_frame = cv2.threshold(gray_frame, 120, 255, cv2.THRESH_BINARY)
    edges_frame = cv2.Canny(binary_frame, 100, 200)
    contornos, _ = cv2.findContours(edges_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    quinas = procura_quinas(contornos)
    
    if len(quinas) == 0:
        return [], []
    
    transformadas = []
    for q in quinas:
        homografia, mascara = cv2.findHomography(np.float32(q), np.float32(quinas_alvo), cv2.RANSAC)
        transformadas.append(cv2.warpPerspective(binary_frame, homografia, (binary_alvo.shape[0], binary_alvo.shape[1])))
    novas_quinas, orientacoes = faz_correlacao(transformadas, quinas)
    return novas_quinas, orientacoes

Com a parte de Template Matching devidamente feita, podemos seguir para a estimação de pose, que é realizada pela função "gera_extrinsecos", que recebe as quinas dos alvos de um frame e suas orientações e retorna uma lista contendo as matrizes de parâmetros extrínsecos dos alvos presentes no frame. Inicialmente carregamos os parâmetros intrínsecos da câmera e criamos uma lista vazia. Para cada lista de quinas na lista recebida, criamos um objeto 3D cuja construção depende da orientação do alvo (o que muda é basicamente a ordem dos pontos, sendo correspondentes a rotação, então são quatro objetos diferentes) e usamos a função "cv2.solvePnP" passando esse objeto, as quinas e os parâmetros intrínsecos.

Com a saída do "cv2.solvePNP", uma matriz de rotação e uma matriz de translação, utilizamos a função "cv2.Rodrigues" para transformar a matriz de rotação em uma matriz 3x3. Com isso podemos montar, finalmente, a matriz de parâmetros intrínsecos. Note que ela possui as linhas 1 e 2 (começa do 0) negativas, isso foi feito para converter o sistema de coordenadas para o sistema de coordenadas do OpenGL, que basicamente consistem em inverter o eixo Y e Z. Também foi necessário transpor a matriz para que ela pudesse ser utilizada. Todas essas matrizes criadas são inseridas na lista vazia criada anteriormente que, por sua vez, é retornada.

Fontes: <br>
https://www.pythonpool.com/opencv-solvepnp/<br>
https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga549c2075fac14829ff4a58bc931c033d<br>
https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga61585db663d9da06b68e70cfbf6a1eac<br>
https://stackoverflow.com/questions/18637494/camera-position-in-world-coordinate-from-cvsolvepnp<br>

In [4]:
def gera_extrinsecos(novas_quinas, orientacoes):
    matriz_intri = np.array([[fx , 0., cx],
                            [0., fy, cy],
                            [0., 0., 1.]])
    distCoeffs = np.array([0.10892, -0.35851, -0.00736, -0.00382,  0.00000])
    extrinsecos = []
    
    for q in range(len(novas_quinas)):
        if(orientacoes[q] == 90):
            objeto = np.array([[ 1, -1, 0], [ 1, 1, 0], [-1, 1, 0], [-1, -1, 0]], dtype="float32")
        elif(orientacoes[q] == 180):
            objeto = np.array([[ 1, 1, 0], [-1, 1, 0], [-1, -1, 0], [ 1, -1, 0]], dtype="float32")
        elif(orientacoes[q] == 270):
            objeto = np.array([[-1, 1, 0], [-1, -1, 0], [ 1, -1, 0], [ 1, 1, 0]], dtype="float32")
        else:
            objeto = np.array([[-1, -1, 0], [ 1, -1, 0], [ 1, 1, 0], [-1, 1, 0]], dtype="float32")
    
        _, rvecs, tvecs = cv2.solvePnP(objeto, np.float32(novas_quinas[q]), matriz_intri, distCoeffs,
                                       flags=cv2.SOLVEPNP_ITERATIVE)
        rotm = cv2.Rodrigues(rvecs)[0]
        m_extrinsecos = np.array([[ rotm[0][0],  rotm[0][1],  rotm[0][2],  tvecs[0]], 
                                  [-rotm[1][0], -rotm[1][1], -rotm[1][2], -tvecs[1]], 
                                  [-rotm[2][0], -rotm[2][1], -rotm[2][2], -tvecs[2]],
                                  [        0.0,         0.0,         0.0,      1.0]])
    
        m_extrinsecos = np.transpose(m_extrinsecos)
        extrinsecos.append(m_extrinsecos)
    return extrinsecos

# OpenGL
Agora vamos adentrar nas funções relacionadas ao OpenGL, o trecho de código abaixo é o que foi disponibilizado para a realização do trabalho. A única diferença se dá no cálculo das variáveis "fovy" e "aspect", que foram alterados para seguir o exemplo do slide da aula de OpenGL.

In [5]:
def initOpenGL(dimensions):

    (width, height) = dimensions
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)

    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
 
    fovy = 2.0*np.arctan(0.5*480.0/fy)*180.0/np.pi
    aspect = (width*fy)/(height*fx)
    gluPerspective(fovy, aspect, 0.1, 100.0)

Seguindo as especificações do trabalho, foi necessário criar uma função que desenhasse, em tempo real, as marcações dos alvos encontrados nos frames do vídeo. Essa função recebe uma lista, que contém as listas de quinas de cada alvo encontrado, e faz um desenho ao seu redor. É necessário desativar a profundidade, para que o desenho fique por cima do fundo criado, que será abordado em breve, mudamos o modo para a projeção, para que as matrizes sejam aplicadas "na câmera", carregamos a identidade e utilizamos a projeção ortográfica paralela 2D, de forma que fique no formato do vídeo. Desativamos a textura, para que a cor da linha de marcação não afete a imagem de fundo, aumentamos a espessura da linha e mudamos sua cor para verde.

Podemos então mudar para o modo "modelview", para aplicar mudanças no objeto, percorremos todas as quinas de cada alvo, criando um quadrilátero usando "GL_LINE_LOOP". Como o OpenGL funciona como uma máquina de estados, tudo que foi alterado deve ser desfeito na mesma ordem, portanto voltamos a cor inicial, diminuímos a espessura da linha, habilitamos a textura e a profundidade. Um ponto importante sobre esta documentação é que os próximos usos dessas funções relacionadas a biblioteca OpenGL não serão tão detalhados quanto esse.

Fontes:<br>
http://linuxfocus.org/English/January1998/article17.html<br>
https://open.gl/drawing<br>

In [6]:
def desenha_borda_alvos(quinas_alvos):
    glDisable(GL_DEPTH_TEST)
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0, 640, 0, 480)
    
    glDisable(GL_TEXTURE_2D)
    glLineWidth(2.0)
    glColor3f(0.0, 1.0, 0.0)
    
    glMatrixMode(GL_MODELVIEW)
    
    for j in range(len(quinas_alvos)):
        glBegin(GL_LINE_LOOP)
        for k in range(len(quinas_alvos[j])):
            glVertex2i(quinas_alvos[j][k][0], 480 - quinas_alvos[j][k][1])
        glEnd()
    
    glColor3f(1.0, 1.0, 1.0)
    glLineWidth(1.0)
    glEnable(GL_TEXTURE_2D)
    glEnable(GL_DEPTH_TEST)

Na célula abaixo temos o método que coordena grande parte do trabalho em relação a biblioteca OpenGL, além de ser a integração entre o Template Matching, a detecção de pose e a renderização dos objetos 3D. A função recebe o objeto do Pikachu e o número do frame atual, ou seja, tudo é feito em "tempo real". Inicialmente é chamada a função de localização de alvos, que retorna as quinas dos alvos e a orientação de cada um, e em seguida é feito o desenho das bordas dos alvos.

Vamos então para o modo de projeção, nele carregamos a matriz identidade e a perspectiva, que é igual a da função "initOpenGL". Para desenhar o Pikachu e os outros objetos nos alvos, precisamos saber a pose deles, que é obtida utilizando-se a função "gera_extrinsecos", que retorna uma lista de matrizes de parâmetros extrínsecos de cada alvo.

Para cada uma dessas matrizes, mudamos para o modo "modelview", carregamos a matriz de parâmetros extrínsecos, desativamos a textura para podermos desenhar o cubo (transladando-o meio cubo acima para que a base dele fique no plano do fundo) e a reta com o cone na frente. Para que pudéssemos posicionar a seta (reta junto com o cone), foi necessário transladar e rotacionar com base nos parâmetros extrínsecos adicionados e fazer toda a rotação de volta, para que pudéssemos desenhar o Pikachu na posição correta.

Por fim, reativamos a textura, já que o Pikachu possui textura, e verificamos onde o primeiro ponto do alvo em questão está em relação a imagem do frame. Isso foi feito com o intuito de rotacionar cada Pìkachu no sentido correto, de acordo com a numeração dos alvos. Já que o alvo que deveria possuir uma rotação diferente está sempre mais no canto superior direito da imagem, foram utilizados X > 300 e Y < 150 como limite para definir a rotação.


In [7]:
def object3D(obj, frame_atual):
    global angle
    quinas_alvos, orientacao = localiza_alvos(frames[frame_atual])
    desenha_borda_alvos(quinas_alvos)
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    fovy = 2.0*np.arctan(0.5*480.0/fy)*180.0/np.pi
    aspect = (640*fy)/(480*fx)
    gluPerspective(fovy, aspect, 0.1, 100.0)
    
    extrinsecos = gera_extrinsecos(quinas_alvos, orientacao)
    
    for j in range(len(extrinsecos)):
        glMatrixMode(GL_MODELVIEW)
        glLoadMatrixd(extrinsecos[j])
                
        glDisable(GL_TEXTURE_2D)
        glTranslate(0,0,1.5)
        glutWireCube(3.0)        
        glTranslate(0,0,-1.5)
        
        glLineWidth(2.0)
        glBegin(GL_LINES)
        glVertex2f(0, 0.0)
        glVertex2f(-3, 0.0)
        glEnd()
        glLineWidth(1.0)
        
        glTranslate(-2.5, 0, 0)
        glRotate(90,1,0,0)
        glRotate(-90,0,1,0)
        
        glutWireCone(0.3, 2, 8, 8)
        
        glRotate(90,0,1,0)
        glRotate(-90,1,0,0)
        glTranslate(2.5, 0, 0)
        
        glEnable(GL_TEXTURE_2D)
        
        if quinas_alvos[j][0][0] > 300 and quinas_alvos[j][0][1] < 150:
            glRotatef(-angle, 0, 0, 1)
        else:
            glRotatef(angle, 0, 0, 1)
            
        angle += 1
        glCallList(obj.gl_list)
    

A função da célula abaixo recebe o número do frame atual e o coloca como fundo, para isso utilizamos a imagem como uma textura. Mas antes disso precisamos desativar a profundidade e a máscara de profundidade, além de inverter a imagem devido ao sistema de coordenadas da biblioteca OpenGL, e mudar o sistema de cores de BGR (padrão da biblioteca OpenCV) para o RGB (padrão da biblioteca OpenGL). Uma referência importante a ser citada é este link: https://stackoverflow.com/questions/32210107/opencv-python-opengl-texture , que foi de extrema importância para o trabalho como um todo.

In [8]:
def desenha_fundo(frame_atual):
    glDisable(GL_DEPTH_TEST)
    glDepthMask(GL_FALSE)
    
    f = frames[frame_atual].copy()
    f = cv2.flip(f, 0)
    f = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
    glBindTexture(GL_TEXTURE_2D, glGenTextures(1))
    glTexImage2D(GL_TEXTURE_2D, 
      0, 
      GL_RGB, 
      640,480,
      0,
      GL_RGB, 
      GL_UNSIGNED_BYTE, 
      f)
    
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0, 640, 0, 480)
    
    glMatrixMode(GL_MODELVIEW)
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 0.0); glVertex2f(0.0, 0.0)
    glTexCoord2f(1.0, 0.0); glVertex2f(640.0, 0.0)
    glTexCoord2f(1.0, 1.0); glVertex2f(640.0, 480.0)
    glTexCoord2f(0.0, 1.0); glVertex2f(0.0, 480.0)
    glEnd()
    
    glDepthMask(GL_TRUE)
    glEnable(GL_DEPTH_TEST)

A função abaixo foi dada e as únicas mudanças foram incluir a chamada da função que desenha o fundo, a remoção do carregamento do modelo 3D do Pikachu, visto que estava deixando o processo muito lento, e a gerência dos frames do vídeo. Um ponto a ser observado é que quando o vídeo acaba, a janela é imediatamente fechada.

In [9]:
def displayCallback():
    global frame_atual
    global obj

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    glEnable(GL_TEXTURE_2D)

    desenha_fundo(frame_atual)
    object3D(obj, frame_atual)
    
    frame_atual += 1
    if frame_atual >= len(frames[:1100]):
        glutLeaveMainLoop()
    glutSwapBuffers()

Novamente, o conteúdo da célula abaixo foi dado, a única mudança é que o carregamento do modelo 3D do Pikachu foi transferido para cá, já que só é necessário carregar uma única vez, aumentando a velocidade e diminuindo o processamento necessário.

In [10]:
def idleCallback():

    glutPostRedisplay()
    
if __name__ == '__main__':

    dimensions = (640, 480)
    glutInit()
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE | GLUT_DEPTH)
    glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
    glutInitWindowSize(*dimensions)
    window = glutCreateWindow(b'Realidade Aumentada')
    
    initOpenGL(dimensions)
    # carregar o modelo 3D do Pikachu
    obj = OBJ("Pikachu.obj", swapyz=True)
    
    glutDisplayFunc(displayCallback)
    glutIdleFunc(idleCallback)
    
    glutMainLoop()

C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
